# Finetuning of the pretrained Japanese BERT model

Finetune the pretrained model to solve multi-class classification problems.  
This notebook requires the following objects:
- trained sentencepiece model (model and vocab files)
- pretraiend Japanese BERT model

Dataset is livedoor ニュースコーパス in https://www.rondhuit.com/download.html.  
We make test:dev:train = 2:2:6 datasets.

Results:

- Full training data
  - BERT with SentencePiece
    ```
                    precision    recall  f1-score   support

    dokujo-tsushin       0.98      0.94      0.96       178
      it-life-hack       0.96      0.97      0.96       172
     kaden-channel       0.99      0.98      0.99       176
    livedoor-homme       0.98      0.88      0.93        95
       movie-enter       0.96      0.99      0.98       158
            peachy       0.94      0.98      0.96       174
              smax       0.98      0.99      0.99       167
      sports-watch       0.98      1.00      0.99       190
        topic-news       0.99      0.98      0.98       163

         micro avg       0.97      0.97      0.97      1473
         macro avg       0.97      0.97      0.97      1473
      weighted avg       0.97      0.97      0.97      1473
    ```
  - sklearn GradientBoostingClassifier with MeCab
    ```
                      precision    recall  f1-score   support

    dokujo-tsushin       0.89      0.86      0.88       178
      it-life-hack       0.91      0.90      0.91       172
     kaden-channel       0.90      0.94      0.92       176
    livedoor-homme       0.79      0.74      0.76        95
       movie-enter       0.93      0.96      0.95       158
            peachy       0.87      0.92      0.89       174
              smax       0.99      1.00      1.00       167
      sports-watch       0.93      0.98      0.96       190
        topic-news       0.96      0.86      0.91       163

         micro avg       0.92      0.92      0.92      1473
         macro avg       0.91      0.91      0.91      1473
      weighted avg       0.92      0.92      0.91      1473
    ```

- Small training data (1/5 of full training data)
  - BERT with SentencePiece
    ```
                    precision    recall  f1-score   support

    dokujo-tsushin       0.97      0.87      0.92       178
      it-life-hack       0.86      0.86      0.86       172
     kaden-channel       0.95      0.94      0.95       176
    livedoor-homme       0.82      0.82      0.82        95
       movie-enter       0.97      0.99      0.98       158
            peachy       0.89      0.95      0.92       174
              smax       0.94      0.96      0.95       167
      sports-watch       0.97      0.97      0.97       190
        topic-news       0.94      0.94      0.94       163

         micro avg       0.93      0.93      0.93      1473
         macro avg       0.92      0.92      0.92      1473
      weighted avg       0.93      0.93      0.93      1473
    ```
  - sklearn GradientBoostingClassifier with MeCab
    ```
                    precision    recall  f1-score   support

    dokujo-tsushin       0.82      0.71      0.76       178
      it-life-hack       0.86      0.88      0.87       172
     kaden-channel       0.91      0.87      0.89       176
    livedoor-homme       0.67      0.63      0.65        95
       movie-enter       0.87      0.95      0.91       158
            peachy       0.70      0.78      0.73       174
              smax       1.00      1.00      1.00       167
      sports-watch       0.87      0.95      0.91       190
        topic-news       0.92      0.82      0.87       163

         micro avg       0.85      0.85      0.85      1473
         macro avg       0.85      0.84      0.84      1473
      weighted avg       0.86      0.85      0.85      1473
    ```

In [1]:
import configparser
import glob
import os
import pandas as pd
import subprocess
import sys
import tarfile 
from urllib.request import urlretrieve

CURDIR = os.getcwd()
CONFIGPATH = os.path.join(CURDIR, os.pardir, 'config.ini')
config = configparser.ConfigParser()
config.read(CONFIGPATH)

['/home/hiroki-iida/work/bert-japanese/notebook/../config.ini']

## Data preparing

You need execute the following cells just once.

In [2]:
FILEURL = config['FINETUNING-DATA']['FILEURL']
FILEPATH = config['FINETUNING-DATA']['FILEPATH']
EXTRACTDIR = config['FINETUNING-DATA']['TEXTDIR']

Download and unzip data(livedoor corpus).

In [3]:
%%time

urlretrieve(FILEURL, FILEPATH)

mode = "r:gz"
tar = tarfile.open(FILEPATH, mode) 
tar.extractall(EXTRACTDIR) 
tar.close()

CPU times: user 1.11 s, sys: 392 ms, total: 1.5 s
Wall time: 2.33 s


Data preprocessing.

In [4]:
def extract_txt(filename):
    with open(filename) as text_file:
        # 0: URL, 1: timestamp
        text = text_file.readlines()[2:]
        text = [sentence.strip() for sentence in text]
        text = list(filter(lambda line: line != '', text))
        return ''.join(text)

In [5]:
categories = [ 
    name for name 
    in os.listdir( os.path.join(EXTRACTDIR, "text") ) 
    if os.path.isdir( os.path.join(EXTRACTDIR, "text", name) ) ]

categories = sorted(categories)

In [6]:
categories

['dokujo-tsushin',
 'it-life-hack',
 'kaden-channel',
 'livedoor-homme',
 'movie-enter',
 'peachy',
 'smax',
 'sports-watch',
 'topic-news']

In [7]:
table = str.maketrans({
    '\n': '',
    '\t': '　',
    '\r': '',
})

In [8]:
%%time

all_text = []
all_label = []

for cat in categories:
    files = glob.glob(os.path.join(EXTRACTDIR, "text", cat, "{}*.txt".format(cat)))
    files = sorted(files)
    body = [ extract_txt(elem).translate(table) for elem in files ]
    label = [cat] * len(body)
    
    all_text.extend(body)
    all_label.extend(label)

CPU times: user 1.16 s, sys: 120 ms, total: 1.28 s
Wall time: 1.28 s


In [9]:
df = pd.DataFrame({'text' : all_text, 'label' : all_label})

In [10]:
df.head()

,text,label
0,友人代表のスピーチ、独女はどうこなしている？もうすぐジューン・ブライドと呼ばれる６月。独女の...,dokujo-tsushin
1,ネットで断ち切れない元カレとの縁携帯電話が普及する以前、恋人への連絡ツールは一般電話が普通だ...,dokujo-tsushin
2,相次ぐ芸能人の“すっぴん”披露 その時、独女の心境は？「男性はやっぱり、女性の“すっぴん”が...,dokujo-tsushin
3,ムダな抵抗！？ 加齢の現実ヒップの加齢による変化は「たわむ→下がる→内に流れる」、バストは「...,dokujo-tsushin
4,税金を払うのは私たちなんですけど！6月から支給される子ども手当だが、当初は子ども一人当たり月...,dokujo-tsushin


In [11]:
df = df.sample(frac=1, random_state=23).reset_index(drop=True)

In [12]:
df.head()

,text,label
0,新記録でロンドンに乗り込む“バタフライの女王”加藤ゆか3日に行われた競泳の日本選手権で、女子...,sports-watch
1,家電チャンネルの記事も配信！向かうところ敵なしのスマホアプリ「ITニュース by lived...,kaden-channel
2,彼にあげたい韓国メンズコスメ、韓流俳優のような美肌男へ！年末の大イベント、クリスマスまであと...,peachy
3,快適なスマホライフのための必須アプリ「マトリックス レボリューションズ」(c)Warner ...,livedoor-homme
4,独女と上司の気になる関係人事異動の多い春は、職場の人間関係の悩みも増える時期。『an・an』...,dokujo-tsushin


Save data as tsv files.  
test:dev:train = 2:2:6. To check the usability of finetuning, we also prepare sampled training data (1/5 of full training data).

In [13]:
df[:len(df) // 5].to_csv( os.path.join(EXTRACTDIR, "test.tsv"), sep='\t', index=False)
df[len(df) // 5:len(df)*2 // 5].to_csv( os.path.join(EXTRACTDIR, "dev.tsv"), sep='\t', index=False)
df[len(df)*2 // 5:].to_csv( os.path.join(EXTRACTDIR, "train.tsv"), sep='\t', index=False)

### 1/5 of full training data.
# df[:len(df) // 5].to_csv( os.path.join(EXTRACTDIR, "test.tsv"), sep='\t', index=False)
# df[len(df) // 5:len(df)*2 // 5].to_csv( os.path.join(EXTRACTDIR, "dev.tsv"), sep='\t', index=False)
# df[len(df)*2 // 5:].sample(frac=0.2, random_state=23).to_csv( os.path.join(EXTRACTDIR, "train.tsv"), sep='\t', index=False)

## Finetune pre-trained model

It will take a lot of hours to execute the following cells on CPU environment.  
You can also use colab to recieve the power of TPU. You need to uplode the created data onto your GCS bucket.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1zZH2GWe0U-7GjJ2w2duodFfEUptvHjcx)

In [14]:
PRETRAINED_MODEL_PATH = '../model/asahi_output/model.ckpt-1300000'
FINETUNE_OUTPUT_DIR = '../model/livedoor_asahi_output'

In [15]:
%%time
# It will take many hours on CPU environment.

!python3 ../src/run_classifier.py \
  --task_name=livedoor \
  --do_train=true \
  --do_eval=true \
  --data_dir=../data/livedoor \
  --model_file=../model/asahi.model \
  --vocab_file=../model/asahi.vocab \
  --init_checkpoint={PRETRAINED_MODEL_PATH} \
  --max_seq_length=512 \
  --train_batch_size=4 \
  --learning_rate=2e-5 \
  --num_train_epochs=10 \
  --output_dir={FINETUNE_OUTPUT_DIR}

Loaded a trained SentencePiece model.
INFO:tensorflow:Using config: {'_model_dir': '../model/livedoor_asahi_output', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f57a301a860>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=1000, num_shards=8, num_cores_per_replica=None, per_host_input_for_training=3, t

INFO:tensorflow:***** Running training *****
INFO:tensorflow:  Num examples = 4421
INFO:tensorflow:  Batch size = 4
INFO:tensorflow:  Num steps = 11052
Instructions for updating:
Use `tf.data.experimental.map_and_batch(...)`.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running train on CPU
INFO:tensorflow:*** Features ***
INFO:tensorflow:  name = input_ids, shape = (4, 512)
INFO:tensorflow:  name = input_mask, shape = (4, 512)
INFO:tensorflow:  name = is_real_example, shape = (4,)
INFO:tensorflow:  name = label_ids, shape = (4,)
INFO:tensorflow:  name = segment_ids, shape = (4, 512)
INFO:tensorflow:**** Trainable Variables ****
INFO:tensorflow:  name = bert/embeddings/word_embeddings:0, shape = (32000, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/token_type_embeddings:0, shape = (2, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/position_embeddings:0, shape = (512, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/beta:0

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
2019-04-02 11:20:24.820673: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX
2019-04-02 11:20:25.665978: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:964] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-04-02 11:20:25.667161: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1432] Found device 0 with properties: 
name: GeForce GTX 1080 major: 6 minor: 1 memoryClockRate(GHz): 1.7335
pciBusID: 0000:09:00.0
totalMemory: 7.93GiB freeMemory: 7.82GiB
2019-04-02 11:20:25.667189: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1511] Adding visible gpu devices: 0
2019-04-02 11:20:26.237451: I tensorflow/core/common_runtime/gpu/gpu_device.cc:982] Device interconnect Stream

INFO:tensorflow:global_step/sec: 2.14171
INFO:tensorflow:examples/sec: 8.56684
INFO:tensorflow:global_step/sec: 2.14067
INFO:tensorflow:examples/sec: 8.5627
INFO:tensorflow:global_step/sec: 2.14119
INFO:tensorflow:examples/sec: 8.56476
INFO:tensorflow:global_step/sec: 2.14177
INFO:tensorflow:examples/sec: 8.56708
INFO:tensorflow:global_step/sec: 2.14124
INFO:tensorflow:examples/sec: 8.56497
INFO:tensorflow:Saving checkpoints for 8000 into ../model/livedoor_asahi_output/model.ckpt.
INFO:tensorflow:global_step/sec: 1.91379
INFO:tensorflow:examples/sec: 7.65516
INFO:tensorflow:global_step/sec: 2.14072
INFO:tensorflow:examples/sec: 8.5629
INFO:tensorflow:global_step/sec: 2.1406
INFO:tensorflow:examples/sec: 8.56238
INFO:tensorflow:global_step/sec: 2.13693
INFO:tensorflow:examples/sec: 8.54771
INFO:tensorflow:global_step/sec: 2.13942
INFO:tensorflow:examples/sec: 8.55766
INFO:tensorflow:global_step/sec: 2.13943
INFO:tensorflow:examples/sec: 8.55773
INFO:tensorflow:global_step/sec: 2.13967
I

INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: dev-2
INFO:tensorflow:tokens: [CLS] ▁ 露 骨 すぎる !? サム ス ン が i p ho ne は 古い と アン チ キャンペーン を展開 【 話題 】 サム ス ン が 行っている f ac e bo o k で の キャンペーン が 露 骨 だと 話題 になっている 。 サム ス ン は 自社 の スマートフォン 「 ga la x y ▁ s ▁ i i 」 は 最先端 だが アップル 社の 「 i p ho ne 」 はもう 古い と 伝える 画像 を掲載 している 。「 オール ド スクール 」 と 表現 された i p ho ne のそばに は 糸 電話 や 古い タイプの 携帯電話 が置かれ ている 皮肉 たっぷり な 出来 栄 え だ 。 この キャンペーン について 、「 サム ス ン は アップル を 羨 まし が っていて 、 やっている ことが 子ども だ 」 や 「 サム ス ン の 商品 は 好き だが さすがに やり すぎ 」 という声 が あがっている 。 あまりに 露 骨 な この 広告 は もはや ジョー ク だ 。 s am s un g ▁ c on t in u es ▁ w i th ▁ an t i - a p p le ▁ r an t ▁ on ▁ f ac e bo o k , ▁ sa y s ▁ i p ho ne ▁ is ▁ “ ol d ▁ s c ho ol ” ■ 関連 記事 サム ス ン が アップル に 逆転 勝訴 ! ▁ オーストラリア で の販売 差し止め 取り消し ■ 関連 記事 ・ 【 記事 連動 】 音 の 編集 講座 「 音 量の 合わせ 方 」 【 ビデオ s al on 】 ・ k d di が i p ho ne 向け アプリ 「 a u お客 さま サポート 」 ▁ を リ リース 【 話題 】 ・ 電子 書籍 市場 を こ っ そり 支える の は 女の子 の エ ッチ な 願望 ? 【 話題 】 ・ 連載 ● 極 ・ ト 書き 一行 の カット 割り ! ▁第 6 回 【 ビデオ s al on 】 ・ 白 物 家電 は やっ

INFO:tensorflow:***** Running evaluation *****
INFO:tensorflow:  Num examples = 1473 (1473 actual, 0 padding)
INFO:tensorflow:  Batch size = 8
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running eval on CPU
INFO:tensorflow:*** Features ***
INFO:tensorflow:  name = input_ids, shape = (?, 512)
INFO:tensorflow:  name = input_mask, shape = (?, 512)
INFO:tensorflow:  name = is_real_example, shape = (?,)
INFO:tensorflow:  name = label_ids, shape = (?,)
INFO:tensorflow:  name = segment_ids, shape = (?, 512)
INFO:tensorflow:**** Trainable Variables ****
INFO:tensorflow:  name = bert/embeddings/word_embeddings:0, shape = (32000, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/token_type_embeddings:0, shape = (2, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/position_embeddings:0, shape = (512, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNor

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-04-02-03:48:08
INFO:tensorflow:Graph was finalized.
2019-04-02 12:48:08.487933: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1511] Adding visible gpu devices: 0
2019-04-02 12:48:08.487992: I tensorflow/core/common_runtime/gpu/gpu_device.cc:982] Device interconnect StreamExecutor with strength 1 edge matrix:
2019-04-02 12:48:08.488012: I tensorflow/core/common_runtime/gpu/gpu_device.cc:988]      0 
2019-04-02 12:48:08.488027: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1001] 0:   N 
2019-04-02 12:48:08.488246: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1115] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 7544 MB memory) -> physical GPU (device: 0, name: GeForce GTX 1080, pci bus id: 0000:09:00.0, compute capability: 6.1)
INFO:tensorflow:Restoring parameters from ../model/livedoor_asahi_output/model.ckpt-11052
INFO:tensorflow:Running local_init_op.
INFO:te

## Predict using the finetuned model

Let's predict test data using the finetuned model.  

In [16]:
import sys
sys.path.append("../src")

import tokenization_sentencepiece as tokenization
from run_classifier import LivedoorProcessor
from run_classifier import model_fn_builder
from run_classifier import file_based_input_fn_builder
from run_classifier import file_based_convert_examples_to_features
from utils import str_to_value

In [17]:
sys.path.append("../bert")

import modeling
import optimization
import tensorflow as tf

In [18]:
import configparser
import json
import glob
import os
import pandas as pd
import tempfile

bert_config_file = tempfile.NamedTemporaryFile(mode='w+t', encoding='utf-8', suffix='.json')
bert_config_file.write(json.dumps({k:str_to_value(v) for k,v in config['BERT-CONFIG'].items()}))
bert_config_file.seek(0)
bert_config = modeling.BertConfig.from_json_file(bert_config_file.name)

In [19]:
output_ckpts = glob.glob("{}/model.ckpt*data*".format(FINETUNE_OUTPUT_DIR))
latest_ckpt = sorted(output_ckpts)[-1]
FINETUNED_MODEL_PATH = latest_ckpt.split('.data-00000-of-00001')[0]

In [30]:
class FLAGS(object):
    '''Parameters.'''
    def __init__(self):
        self.model_file = "../model/asahi.model"
        self.vocab_file = "../model/asahi.vocab"
        self.do_lower_case = True
        self.use_tpu = False
        self.output_dir = "/dummy"
        self.data_dir = "../data/livedoor"
        self.max_seq_length = 512
        self.init_checkpoint = FINETUNED_MODEL_PATH
        self.predict_batch_size = 4
        
        # The following parameters are not used in predictions.
        # Just use to create RunConfig.
        self.master = None
        self.save_checkpoints_steps = 1
        self.iterations_per_loop = 1
        self.num_tpu_cores = 1
        self.learning_rate = 0
        self.num_warmup_steps = 0
        self.num_train_steps = 0
        self.train_batch_size = 0
        self.eval_batch_size = 0

In [31]:
FLAGS = FLAGS()

In [32]:
processor = LivedoorProcessor()
label_list = processor.get_labels()

In [33]:
tokenizer = tokenization.FullTokenizer(
    model_file=FLAGS.model_file, vocab_file=FLAGS.vocab_file,
    do_lower_case=FLAGS.do_lower_case)

tpu_cluster_resolver = None

is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2

run_config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    master=FLAGS.master,
    model_dir=FLAGS.output_dir,
    save_checkpoints_steps=FLAGS.save_checkpoints_steps,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=FLAGS.iterations_per_loop,
        num_shards=FLAGS.num_tpu_cores,
        per_host_input_for_training=is_per_host))

Loaded a trained SentencePiece model.


In [34]:
model_fn = model_fn_builder(
    bert_config=bert_config,
    num_labels=len(label_list),
    init_checkpoint=FLAGS.init_checkpoint,
    learning_rate=FLAGS.learning_rate,
    num_train_steps=FLAGS.num_train_steps,
    num_warmup_steps=FLAGS.num_warmup_steps,
    use_tpu=FLAGS.use_tpu,
    use_one_hot_embeddings=FLAGS.use_tpu)


estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=FLAGS.use_tpu,
    model_fn=model_fn,
    config=run_config,
    train_batch_size=FLAGS.train_batch_size,
    eval_batch_size=FLAGS.eval_batch_size,
    predict_batch_size=FLAGS.predict_batch_size)

INFO:tensorflow:Using config: {'_model_dir': '/dummy', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f25ccd87320>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=1, num_shards=1, num_cores_per_replica=None, per_host_input_for_training=3, tpu_job_name=None, initial_infeed_sleep_secs=None, input_partition_di

In [35]:
predict_examples = processor.get_test_examples(FLAGS.data_dir)
predict_file = tempfile.NamedTemporaryFile(mode='w+t', encoding='utf-8', suffix='.tf_record')

file_based_convert_examples_to_features(predict_examples, label_list,
                                        FLAGS.max_seq_length, tokenizer,
                                        predict_file.name)

predict_drop_remainder = True if FLAGS.use_tpu else False

predict_input_fn = file_based_input_fn_builder(
    input_file=predict_file.name,
    seq_length=FLAGS.max_seq_length,
    is_training=False,
    drop_remainder=predict_drop_remainder)

INFO:tensorflow:Writing example 0 of 1473
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: test-1
INFO:tensorflow:tokens: [CLS] ▁ 新記録 で ロンドン に乗り 込む “ バタフライ の 女王 ” 加藤 ゆか 3 日に 行われた 競泳 の 日本選手権 で 、 女子 100 メートル バタフライ の 加藤 ゆか ( 25 歳 ) が 2 大会連続 の 五輪 出場を決めた 。 57 秒 77 と 、 自身が 持つ 日本 新 記録を更新 して の 五輪 切符 ゲ ット だ 。 前回 大会の 北京五輪 選考 では ガ チ ガ チ に 緊張 していたという 加藤 は 、 幾 多 の経験 を得て 、 強い 精神 力を 培った 。 記録 更新 で の 五輪出場 権 獲得 に 、 爽 やかな 笑顔で 喜び を 爆発 させた 。 百 花 繚 乱 の日本 女子 競泳 界 の中でも 、 加藤 は 美 女 スイ マー の 筆頭 に 数え られる 。 目 鼻 立ち の 整った 顔に 、 白く 透 き 通 るような 肌 、 そして 鍛え あげられ た アスリート の 肉体 に 、 ネット 上で の人気 も 上 々 だ 。 前回の 北京五輪 では 予選 敗退 で 悔し 涙 を 呑 んだ 加藤 。 4 年間の 濃 密 な 時間 を経て 、 速く 、 より 美しく なった 「 バタフライ の 女王 」 が 、 ロンドン の プール を 沸 かす 。 ・ 加藤 ゆか ▁ フォト [SEP]
INFO:tensorflow:input_ids: 4 10 26759 18 4805 4989 1137 1400 24128 9 15256 1388 1823 18244 28 191 5169 25737 9 27637 18 7 455 221 227 24128 9 1823 18244 15 180 152 14 12 25 27800 9 1669 25962 8 1778 376 1619 20 7 9162 6317 103 104 31163 69 9 1669 12528 2829 1453 40 8 1748 6911 18128 4566 52 9

INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [36]:
result = estimator.predict(input_fn=predict_input_fn)

In [37]:
%%time
# It will take a few hours on CPU environment.

result = list(result)

INFO:tensorflow:Could not find trained model in model_dir: /dummy, running initialization to predict.
Instructions for updating:
Use `tf.data.experimental.map_and_batch(...)`.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running infer on CPU
INFO:tensorflow:*** Features ***
INFO:tensorflow:  name = input_ids, shape = (?, 512)
INFO:tensorflow:  name = input_mask, shape = (?, 512)
INFO:tensorflow:  name = is_real_example, shape = (?,)
INFO:tensorflow:  name = label_ids, shape = (?,)
INFO:tensorflow:  name = segment_ids, shape = (?, 512)
INFO:tensorflow:**** Trainable Variables ****
INFO:tensorflow:  name = bert/embeddings/word_embeddings:0, shape = (32000, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/token_type_embeddings:0, shape = (2, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/position_embeddings:0, shape = (512, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow

INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT

INFO:tensorflow:  name = bert/encoder/layer_8/intermediate/dense/bias:0, shape = (3072,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/output/dense/kernel:0, shape = (3072, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder

In [38]:
result[:2]

[{'probabilities': array([0.08920389, 0.09514713, 0.16614906, 0.08158392, 0.06454945,
         0.1859993 , 0.07285181, 0.12818837, 0.11632711], dtype=float32)},
 {'probabilities': array([0.08920389, 0.09514713, 0.16614906, 0.08158392, 0.06454945,
         0.18599929, 0.07285181, 0.12818837, 0.11632711], dtype=float32)}]

Read test data set and add prediction results.

In [39]:
import pandas as pd

In [40]:
test_df = pd.read_csv("../data/livedoor/test.tsv", sep='\t')

In [41]:
test_df['predict'] = [ label_list[elem['probabilities'].argmax()] for elem in result ]

In [42]:
test_df.head()

,text,label,predict
0,新記録でロンドンに乗り込む“バタフライの女王”加藤ゆか3日に行われた競泳の日本選手権で、女子...,sports-watch,peachy
1,家電チャンネルの記事も配信！向かうところ敵なしのスマホアプリ「ITニュース by lived...,kaden-channel,peachy
2,彼にあげたい韓国メンズコスメ、韓流俳優のような美肌男へ！年末の大イベント、クリスマスまであと...,peachy,peachy
3,快適なスマホライフのための必須アプリ「マトリックス レボリューションズ」(c)Warner ...,livedoor-homme,peachy
4,独女と上司の気になる関係人事異動の多い春は、職場の人間関係の悩みも増える時期。『an・an』...,dokujo-tsushin,peachy


In [43]:
sum( test_df['label'] == test_df['predict'] ) / len(test_df)

0.11812627291242363

A littel more detailed check using `sklearn.metrics`.

In [44]:
!pip install -U scikit-learn

    100% |████████████████████████████████| 5.4MB 9.9MB/s eta 0:00:01
  Found existing installation: scikit-learn 0.20.1
    Uninstalling scikit-learn-0.20.1:
      Successfully uninstalled scikit-learn-0.20.1


In [45]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [46]:
print(classification_report(test_df['label'], test_df['predict']))

                precision    recall  f1-score   support

dokujo-tsushin       0.00      0.00      0.00       178
  it-life-hack       0.00      0.00      0.00       172
 kaden-channel       0.00      0.00      0.00       176
livedoor-homme       0.00      0.00      0.00        95
   movie-enter       0.00      0.00      0.00       158
        peachy       0.12      1.00      0.21       174
          smax       0.00      0.00      0.00       167
  sports-watch       0.00      0.00      0.00       190
    topic-news       0.00      0.00      0.00       163

     micro avg       0.12      0.12      0.12      1473
     macro avg       0.01      0.11      0.02      1473
  weighted avg       0.01      0.12      0.02      1473



/home/hiroki-iida/.pyenv/versions/anaconda3-5.1.0/envs/py36_ems/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [47]:
print(confusion_matrix(test_df['label'], test_df['predict']))

[[  0   0   0   0   0 178   0   0   0]
 [  0   0   0   0   0 172   0   0   0]
 [  0   0   0   0   0 176   0   0   0]
 [  0   0   0   0   0  95   0   0   0]
 [  0   0   0   0   0 158   0   0   0]
 [  0   0   0   0   0 174   0   0   0]
 [  0   0   0   0   0 167   0   0   0]
 [  0   0   0   0   0 190   0   0   0]
 [  0   0   0   0   0 163   0   0   0]]


### Simple baseline model.

In [38]:
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [39]:
train_df = pd.read_csv("../../data/livedoor/train.tsv", sep='\t')
dev_df = pd.read_csv("../../data/livedoor/dev.tsv", sep='\t')
test_df = pd.read_csv("../../data/livedoor/test.tsv", sep='\t')

In [40]:
!sudo apt-get install -q -y mecab libmecab-dev mecab-ipadic mecab-ipadic-utf8

Reading package lists...
Building dependency tree...
Reading state information...
libmecab-dev is already the newest version (0.996-1.2ubuntu1).
mecab is already the newest version (0.996-1.2ubuntu1).
mecab-ipadic is already the newest version (2.7.0-20070801+main-1).
mecab-ipadic-utf8 is already the newest version (2.7.0-20070801+main-1).
The following packages were automatically installed and are no longer required:
  libaio1 librados2 librbd1 librdmacm1
Use 'sudo apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 14 not upgraded.


In [41]:
!pip install mecab-python3==0.7

You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import GradientBoostingClassifier
import MeCab

In [43]:
m = MeCab.Tagger("-Owakati")

In [44]:
train_dev_df = pd.concat([train_df, dev_df])

In [45]:
train_dev_xs = train_dev_df['text'].apply(lambda x: m.parse(x))
train_dev_ys = train_dev_df['label']

test_xs = test_df['text'].apply(lambda x: m.parse(x))
test_ys = test_df['label']

In [46]:
vectorizer = TfidfVectorizer(max_features=750)
train_dev_xs_ = vectorizer.fit_transform(train_dev_xs)
test_xs_ = vectorizer.transform(test_xs)

The following set up is not exactly identical to that of BERT because inside Classifier it uses `train_test_split` with shuffle.  
In addition, parameters are not well tuned, however, we think it's enough to check the power of BERT.

In [48]:
%%time

# model = GradientBoostingClassifier(n_estimators=200,
#                                   validation_fraction=len(train_df)/len(dev_df),
#                                   n_iter_no_change=5,
#                                   tol=0.01,
#                                   random_state=23)

### 1/5 of full training data.
model = GradientBoostingClassifier(n_estimators=200,
                                    validation_fraction=len(dev_df)/len(train_df),
                                    n_iter_no_change=5,
                                    tol=0.01,
                                    random_state=23)

model.fit(train_dev_xs_, train_dev_ys)

CPU times: user 2min 56s, sys: 0 ns, total: 2min 56s
Wall time: 2min 56s


In [49]:
print(classification_report(test_ys, model.predict(test_xs_)))

                precision    recall  f1-score   support

dokujo-tsushin       0.89      0.86      0.88       178
  it-life-hack       0.91      0.90      0.91       172
 kaden-channel       0.90      0.94      0.92       176
livedoor-homme       0.79      0.74      0.76        95
   movie-enter       0.93      0.96      0.95       158
        peachy       0.87      0.92      0.89       174
          smax       0.99      1.00      1.00       167
  sports-watch       0.93      0.98      0.96       190
    topic-news       0.96      0.86      0.91       163

     micro avg       0.92      0.92      0.92      1473
     macro avg       0.91      0.91      0.91      1473
  weighted avg       0.92      0.92      0.91      1473



In [50]:
print(confusion_matrix(test_ys, model.predict(test_xs_)))

[[153   4   1   4   2  13   0   1   0]
 [  3 155   6   3   0   1   0   2   2]
 [  0   5 165   0   2   0   1   1   2]
 [  3   4   6  70   4   6   0   1   1]
 [  1   0   1   3 152   1   0   0   0]
 [  7   1   1   2   3 160   0   0   0]
 [  0   0   0   0   0   0 167   0   0]
 [  1   0   0   2   0   0   0 186   1]
 [  3   1   3   5   0   3   0   8 140]]
